# After years of thought it's finally here

In [544]:
import pandas as pd 
import numpy as np
pd.set_option('display.max_rows', 10)

In [545]:
df=pd.read_excel('Pokemon.xlsx')

ditto has been removed from the list
mons with a mega evolution with a new type, have the new type as the 'third type'
mons with an ability like Steelworker or Rocky Payload have their third type as the 1.5 boosted type
(this is for the sake of stab being appreciated against elite types)
However the moves of this third type will still be noted down

#moves that already share stab with the mon are not included, this program is just to find coverage

#the moves included are moves above a power of 65 and above

#except bulldoze, rock tomb, low sweep are included, breaking swipe is included (chilling water, flame charge and trailblaze are not included) (icy wind and electroweb also not included)

#if a mon gets flamethrower, I won't write fire blast or heat wave, and if a mon gets thunderbolt, I won't write thunder, no blizzard if it learns ice beam

#priority moves with low BP have been included, also pivot moves with low bp like flip turn (haven't included normal type priority though)

#multi hit moves will be included (even beat up)

#stored power, avalanche, fling, pursuit, payback, acrobatics, revenge and reversal are also included

#stomping tantrum and high horsepower will be avoided if it already learns earthquake

#superpower will be avoided if it learns close combat

#U-turn is spelled as uturn, X-Scissor is X Scissor

#for mons with technician I'll add moves above 40

#no dream eater, no upper hand, no focus punch, no psywave

#yes two turn moves

#mirror coat, metal burst and counter are typeless

#not including natural gift which gives unlimited coverage


In [546]:
display=pd.concat([df.loc[0:25,'Name':'MonType3'],df.loc[0:25,'Move1':]],axis=1)
display

,Name,MonType1,MonType2,MonType3,Move1,Type1,Move2,Type2,Move3,Type3,...,Move33,Type33,Move34,Type34,Move35,Type35,Move36,Type36,Move37,Type37
0,Abomasnow,Grass,Ice,NaN,Brick Break,Fighting,Earthquake,Ground,Earth Power,Ground,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Absol,Dark,NaN,NaN,Close Combat,Fighting,Iron Tail,Steel,Megahorn,Bug,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Accelgor,Bug,NaN,NaN,Focus Blast,Fighting,Energy Ball,Grass,Sludge Bomb,Poison,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aerodactyl,Rock,Flying,NaN,Aqua Tail,Water,Crunch,Dark,Dragon Claw,Dragon,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aggron,Steel,Rock,NaN,Aqua Tail,Water,Body Press,Fighting,Avalanche,Ice,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,Aromatisse,Fairy,NaN,NaN,Energy Ball,Grass,Flash Cannon,Steel,Psychic,Psychic,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,Articuno,Ice,Flying,NaN,Extrasensory,Psychic,Freeze Dry,Electric,Signal Beam,Bug,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,Articuno-Galar,Psychic,Flying,NaN,Shadow Ball,Ghost,Uturn,Bug,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,Audino,Normal,NaN,Fairy,Dazzling Gleam,Fairy,Drain Punch,Fighting,Flamethrower,Fire,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [547]:
'''df.columns=df.columns.str.replace('Coverage','Type')
df.columns
df.to_excel('Pokemon.xlsx',index=False)'''

"df.columns=df.columns.str.replace('Coverage','Type')\ndf.columns\ndf.to_excel('Pokemon.xlsx',index=False)"

In [548]:
#Type Offense dictionary
#mirror coat and counter are typeless
Off={
    'Typeless':[],
    'Normal':[],
    'Fire':['Grass','Ice','Bug','Steel'],
    'Water':['Fire','Ground','Rock'],
    'Grass':['Water','Ground','Rock'],
    'Electric':['Water','Flying'],
    'Ice':['Grass','Ground','Flying','Dragon'],
    'Fighting':['Normal','Ice','Rock','Dark','Steel'],
    'Poison':['Grass','Fairy'],
    'Ground':['Fire','Electric','Poison','Rock','Steel'],    #ground is good against nearly half dragons so added that
    'Flying':['Grass','Fighting','Bug'],
    'Psychic':['Fighting','Poison'],
    'Bug':['Grass','Psychic','Dark'],
    'Rock':['Fire','Ice','Flying','Bug','Dragon'],   #rock is good against nearly half dragons so added that
    'Ghost':['Psychic','Ghost'],
    'Dragon':['Dragon'],
    'Dark':['Psychic','Ghost'],
    'Steel':['Ice','Rock','Fairy'],
    'Fairy':['Fighting','Dragon','Dark'],
    'nan' :[]
}
#notes: freeze dry is seen as an electric type move to include super effectiveness on water

In [549]:
offdf=pd.Series(Off) #DataFrame doesn't work because all arrays need to be the same length

In [550]:
offdf #this has the indexes as the types, so easy to access with loc

Typeless                          []
Normal                            []
Fire        [Grass, Ice, Bug, Steel]
Water           [Fire, Ground, Rock]
Grass          [Water, Ground, Rock]
                      ...           
Dragon                      [Dragon]
Dark                [Psychic, Ghost]
Steel             [Ice, Rock, Fairy]
Fairy       [Fighting, Dragon, Dark]
nan                               []
Length: 20, dtype: object

In [551]:
#set(np.append((np.append(display['Move1'].unique(),display['Move2'].unique())),display['Move3'].unique()))
#set(display['Move1'].tolist()+display['Move2'].tolist()+display['Move3'].tolist())
#fucking around, better code later

In [552]:
moves_columns=df.columns[df.columns.str.contains('Move', na=False)].tolist()
moves_columns
moves_columns.append('Name') #I append Name, but later I set 

movesdf=df[moves_columns]
#movesdf=movesdf.dropna() #this is bad, this drops all incomplete rows, so the only row left is Ambipom's row which goes till move37
movesdf=movesdf.set_index('Name')   
movesdf


,Move1,Move2,Move3,Move4,Move5,Move6,Move7,Move8,Move9,Move10,...,Move28,Move29,Move30,Move31,Move32,Move33,Move34,Move35,Move36,Move37
Name,,,,,,,,,,,,,,,,,,,,,
Abomasnow,Brick Break,Earthquake,Earth Power,Iron Tail,Low Kick,Rock Slide,Shadow Ball,Body Press,Outrage,Fling,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Absol,Close Combat,Iron Tail,Megahorn,Play Rough,Psycho Cut,Rock Slide,Shadow Claw,Stone Edge,X Scissor,Zen Headbutt,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accelgor,Focus Blast,Energy Ball,Sludge Bomb,Water Shuriken,Knock Off,Pursuit,Drain Punch,Reversal,Venoshock,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aerodactyl,Aqua Tail,Crunch,Dragon Claw,Earthquake,Fire Fang,Ice Fang,Iron Head,Psychic Fangs,Thunder Fang,Dragon Pulse,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aggron,Aqua Tail,Body Press,Avalanche,Crunch,Dragon Claw,Earthquake,Fire Punch,Ice Punch,Low Kick,Outrage,...,Counter,Dig,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zarude,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zebstrika,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zeraora,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [553]:
type_columns=df.columns[df.columns.str.contains('Type', na=False)].tolist() #works out nice cuz you need both montype and nonstab type
type_columns
type_columns.append('Name')
type_df=df[type_columns]
type_df=type_df.set_index('Name')
type_df

,MonType1,MonType2,MonType3,Type1,Type2,Type3,Type4,Type5,Type6,Type7,...,Type28,Type29,Type30,Type31,Type32,Type33,Type34,Type35,Type36,Type37
Name,,,,,,,,,,,,,,,,,,,,,
Abomasnow,Grass,Ice,NaN,Fighting,Ground,Ground,Steel,Fighting,Rock,Ghost,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Absol,Dark,NaN,NaN,Fighting,Steel,Bug,Fairy,Psychic,Rock,Ghost,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accelgor,Bug,NaN,NaN,Fighting,Grass,Poison,Water,Dark,Dark,Fighting,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aerodactyl,Rock,Flying,NaN,Water,Dark,Dragon,Ground,Fire,Ice,Steel,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aggron,Steel,Rock,NaN,Water,Fighting,Ice,Dark,Dragon,Ground,Fire,...,Typeless,Ground,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zarude,Dark,Grass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zebstrika,Electric,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zeraora,Electric,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [554]:
#Temporary code, apparently one of the moves is named 'Dark', which means I messed up somwehere, so I'm trying to find the index of the place I messed up so that I can edit it in Excel
#once I fix it, this code will be useless.
indices=np.where(movesdf=='Dark')
print(indices)

#row_indices, col_indices = indices[0], indices[1]
#print(row_indices,col_indices)

#so apparently on row 32, columns 11 and 12, there's a move named Dark

'''
movesdf.iloc[32,11]
movesdf.iloc[32,12]

print(df.loc[32,'Move12'])
print(df.loc[32,'Move13'])

print(df.loc[32,'Name'])
'''
#Okay it's Barraskewda, time to fix it

(array([], dtype=int64), array([], dtype=int64))


"\nmovesdf.iloc[32,11]\nmovesdf.iloc[32,12]\n\nprint(df.loc[32,'Move12'])\nprint(df.loc[32,'Move13'])\n\nprint(df.loc[32,'Name'])\n"

In [555]:
#exploration code

#moveslist=pd.concat([df['Move1'],df['Move2'],df['Move3']]).reset_index() #without this, every index has values of all the different moves
#moveslist.columns=['old_repeated_index','moves']
#moveslist.dropna()
#moveslist[moveslist['old_repeated_index']==2]
#all the above code was a waste after I discovered ignore_index=True


In [556]:
#moveslist=pd.concat([df['Move1'],df['Move2'],df['Move3'],df['Move4'],df['Move5'],df['Move6'],df['Move7'],df['Move8'],df['Move9'],df['Move10'],df['Move11'],df['Move12'],df['Move13'],df['Move14'],df['Move15'],df['Move16'],df['Move17'],df['Move18'],df['Move19'],df['Move20'],df['Move21'],df['Move22'],df['Move23'],df['Move24'],df['Move25'],df['Move26'],df['Move27'],df['Move28'],df['Move29'],df['Move30'],df['Move31'],df['Move32'],df['Move33'],df['Move34'],df['Move35'],df['Move36']], ignore_index=True)
moveslist=pd.concat([movesdf.loc[:, col] for col in movesdf.columns], axis = 0, ignore_index=True)
#moveslist=pd.concat([col for col in movesdf], axis = 0, ignore_index=True)
#all the different moves columns are merged into one column

#moveslist.columns=['old_repeated_index','moves']
#moveslist[moveslist['old_repeated_index']==2]
#moveslist=moveslist.drop('old_repeated_index',axis=1)
#the three lines of code above were a waste after I discovered ignore_index=True

moveslist=moveslist.dropna()


len(moveslist) #this is 17982 without dropping empty cells, and 1127 after dropping
moveslist.nunique() #this is 156

156

create a filter later so that moves that rely on the significantly weaker stat can be removed if desired

so then that means we create a column for a pokemon, which states its dominant offense, and if no dominant offense then it's empty

remember to somehow let counter and mirror coat and metal burst maybe remain unfiltered as strong moves

maybe we should remember when even special attackers have moves like uturn, or physical attackers have moves like volt switch

 do all these complicated stuff with df.query

one filter should also make sure that a type is stab supereffective (but also remember that freeze dry is electric type and needs to be counted as stab)

vectorized method usually means you create another column, like df['result']=df[first] + df[second]

have a string function that inputs something like "Move1" and converts it to "Coverage1" so that you can map moves to their types

have a filter to avoid inconsistent moves like fling, pursuit

In [560]:
type_df

,MonType1,MonType2,MonType3,Type1,Type2,Type3,Type4,Type5,Type6,Type7,...,Type28,Type29,Type30,Type31,Type32,Type33,Type34,Type35,Type36,Type37
Name,,,,,,,,,,,,,,,,,,,,,
Abomasnow,Grass,Ice,NaN,Fighting,Ground,Ground,Steel,Fighting,Rock,Ghost,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Absol,Dark,NaN,NaN,Fighting,Steel,Bug,Fairy,Psychic,Rock,Ghost,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accelgor,Bug,NaN,NaN,Fighting,Grass,Poison,Water,Dark,Dark,Fighting,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aerodactyl,Rock,Flying,NaN,Water,Dark,Dragon,Ground,Fire,Ice,Steel,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aggron,Steel,Rock,NaN,Water,Fighting,Ice,Dark,Dragon,Ground,Fire,...,Typeless,Ground,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zarude,Dark,Grass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zebstrika,Electric,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zeraora,Electric,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [561]:
#try **kwargs, maybe we can input keys as movenames and values as types later
def coverage_function(typelist):
    for i in typelist:
       print(offdf.loc[i])
          
    

In [562]:
#trying the coverage function on a small segment, will do it to the entire database eventually

#type_df.apply(len, axis='columns') #the default is rows btw, which means it applies the function to each row, which is good


#type_df.apply(coverage_function, axis='columns') #We need to decide whether the axis should be columns or rows
'''abomasnow_df=type_df.loc['Abomasnow'].map(Off)
abomasnow_df'''

"abomasnow_df=type_df.loc['Abomasnow'].map(Off)\nabomasnow_df"

In [563]:
'''coverage_df=type_df['MonType1'].map(Off)
coverage_df'''

"coverage_df=type_df['MonType1'].map(Off)\ncoverage_df"

In [564]:
print(Off)

{'Typeless': [], 'Normal': [], 'Fire': ['Grass', 'Ice', 'Bug', 'Steel'], 'Water': ['Fire', 'Ground', 'Rock'], 'Grass': ['Water', 'Ground', 'Rock'], 'Electric': ['Water', 'Flying'], 'Ice': ['Grass', 'Ground', 'Flying', 'Dragon'], 'Fighting': ['Normal', 'Ice', 'Rock', 'Dark', 'Steel'], 'Poison': ['Grass', 'Fairy'], 'Ground': ['Fire', 'Electric', 'Poison', 'Rock', 'Steel'], 'Flying': ['Grass', 'Fighting', 'Bug'], 'Psychic': ['Fighting', 'Poison'], 'Bug': ['Grass', 'Psychic', 'Dark'], 'Rock': ['Fire', 'Ice', 'Flying', 'Bug', 'Dragon'], 'Ghost': ['Psychic', 'Ghost'], 'Dragon': ['Dragon'], 'Dark': ['Psychic', 'Ghost'], 'Steel': ['Ice', 'Rock', 'Fairy'], 'Fairy': ['Fighting', 'Dragon', 'Dark'], 'nan': []}


In [565]:
#full_df_attempt=type_df.replace(Off)
#full_df_attempt

In [566]:
#apparently the above code, using replace instead of applymap is better(more efficient), but for that you need to remove null values, and I want to push 
#myself to see how long I can deal with null values
full_df_attempt=type_df.applymap(lambda x: Off.get(x))
full_df_attempt
#LESSSSGOOOOOO

C:\Users\Srijan Badhya\AppData\Local\Temp\ipykernel_13784\331610883.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  full_df_attempt=type_df.applymap(lambda x: Off.get(x))


,MonType1,MonType2,MonType3,Type1,Type2,Type3,Type4,Type5,Type6,Type7,...,Type28,Type29,Type30,Type31,Type32,Type33,Type34,Type35,Type36,Type37
Name,,,,,,,,,,,,,,,,,,,,,
Abomasnow,"[Water, Ground, Rock]","[Grass, Ground, Flying, Dragon]",None,"[Normal, Ice, Rock, Dark, Steel]","[Fire, Electric, Poison, Rock, Steel]","[Fire, Electric, Poison, Rock, Steel]","[Ice, Rock, Fairy]","[Normal, Ice, Rock, Dark, Steel]","[Fire, Ice, Flying, Bug, Dragon]","[Psychic, Ghost]",...,None,None,None,None,None,None,None,None,None,None
Absol,"[Psychic, Ghost]",None,None,"[Normal, Ice, Rock, Dark, Steel]","[Ice, Rock, Fairy]","[Grass, Psychic, Dark]","[Fighting, Dragon, Dark]","[Fighting, Poison]","[Fire, Ice, Flying, Bug, Dragon]","[Psychic, Ghost]",...,None,None,None,None,None,None,None,None,None,None
Accelgor,"[Grass, Psychic, Dark]",None,None,"[Normal, Ice, Rock, Dark, Steel]","[Water, Ground, Rock]","[Grass, Fairy]","[Fire, Ground, Rock]","[Psychic, Ghost]","[Psychic, Ghost]","[Normal, Ice, Rock, Dark, Steel]",...,None,None,None,None,None,None,None,None,None,None
Aerodactyl,"[Fire, Ice, Flying, Bug, Dragon]","[Grass, Fighting, Bug]",None,"[Fire, Ground, Rock]","[Psychic, Ghost]",[Dragon],"[Fire, Electric, Poison, Rock, Steel]","[Grass, Ice, Bug, Steel]","[Grass, Ground, Flying, Dragon]","[Ice, Rock, Fairy]",...,None,None,None,None,None,None,None,None,None,None
Aggron,"[Ice, Rock, Fairy]","[Fire, Ice, Flying, Bug, Dragon]",None,"[Fire, Ground, Rock]","[Normal, Ice, Rock, Dark, Steel]","[Grass, Ground, Flying, Dragon]","[Psychic, Ghost]",[Dragon],"[Fire, Electric, Poison, Rock, Steel]","[Grass, Ice, Bug, Steel]",...,[],"[Fire, Electric, Poison, Rock, Steel]",None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zarude,"[Psychic, Ghost]","[Water, Ground, Rock]",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Zebstrika,"[Water, Flying]",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Zeraora,"[Water, Flying]",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [567]:
full_df_attempt.loc['Ambipom']

MonType1                                       []
MonType2                                     None
MonType3                                     None
Type1                      [Grass, Fighting, Bug]
Type2            [Normal, Ice, Rock, Dark, Steel]
                            ...                  
Type33           [Fire, Ice, Flying, Bug, Dragon]
Type34                            [Water, Flying]
Type35                     [Grass, Fighting, Bug]
Type36      [Fire, Electric, Poison, Rock, Steel]
Type37                           [Psychic, Ghost]
Name: Ambipom, Length: 40, dtype: object

In [568]:
type(full_df_attempt.loc['Ambipom'])

pandas.core.series.Series

In [569]:
type(type_df.iloc[400,10])

float

In [570]:
type(full_df_attempt.iloc[400,10])

NoneType

In [571]:
(type_df.drop(columns=['MonType1','MonType2','MonType3'])).columns

Index(['Type1', 'Type2', 'Type3', 'Type4', 'Type5', 'Type6', 'Type7', 'Type8',
       'Type9', 'Type10', 'Type11', 'Type12', 'Type13', 'Type14', 'Type15',
       'Type16', 'Type17', 'Type18', 'Type19', 'Type20', 'Type21', 'Type22',
       'Type23', 'Type24', 'Type25', 'Type26', 'Type27', 'Type28', 'Type29',
       'Type30', 'Type31', 'Type32', 'Type33', 'Type34', 'Type35', 'Type36',
       'Type37'],
      dtype='object')

In [572]:
nonstab_columns=(type_df.drop(columns=['MonType1','MonType2','MonType3'])).columns
typeslist=pd.concat([type_df.loc[:, col] for col in nonstab_columns], axis = 0, ignore_index=True)
typeslist

0        Fighting
1        Fighting
2        Fighting
3           Water
4           Water
           ...   
18014         NaN
18015         NaN
18016         NaN
18017         NaN
18018         NaN
Length: 18019, dtype: object

In [573]:
typeslist=typeslist.dropna()
len(typeslist) #this is 17982 without dropping empty cells, and 1127 after dropping

1165

In [574]:
pd.set_option('display.max_rows', None)
smthh=moveslist.drop_duplicates()
smthh.columns=['Move']
smthh=smthh.sort_values()
smthh=(smthh.reset_index()
        .drop(columns='index'))
smthh
#make sure to compare the number of things in this list and the below list to make sure no extra faulty moves got added

,0
0,Acrobatics
1,Aerial Ace
2,Air Slash
3,Alluring Voice
4,Aqua Jet
5,Aqua Tail
6,Assurance
7,Aura Sphere
8,Aurora Beam
9,Avalanche


In [575]:
#if not isinstance(moveslist,pd.DataFrame):
    #moveslist2=moveslist.to_frame(name='Move')
moveslist2=moveslist.to_frame(name='Move')
moveslist2['Type']=typeslist
#moveslist2=moveslist2[:-1] #there's something extremely weird with the last row
moveslist2=(moveslist2.drop_duplicates()
            .sort_values(by='Move')
            .reset_index()
            .drop(columns='index'))
moveslist2


,Move,Type
0,Acrobatics,Flying
1,Aerial Ace,Flying
2,Air Slash,Flying
3,Alluring Voice,Fairy
4,Aqua Jet,Water
5,Aqua Tail,Water
6,Assurance,Dark
7,Aura Sphere,Fighting
8,Aurora Beam,Ice
9,Avalanche,Ice


In [577]:
pd.set_option('display.max_rows', 5)


maybe make a list of slow mons with priority moves later? or just mons with priority moves. So that extreme speed etc can be included

map the moves to the mons, so that maybe later we can just randomly filter the mons with priority moves who generally have coverage against the elite 4

oh wait, maybe we can make a column for mons called, "Priority", and inside that we can put in a list of the priority moves they have, others will be null

so then in our regular filter we can just see good e4 mons, and also particularly see the good e4 mons which have priority

find a way to sort mons such that mons with broaaaaad matching coverage have lower priority, and mons with specific coverage towards the particular e4 are higher priority. 
Cuz like florges has only bug, psychic, grass, and fairy moves, so the moment you get an elite aligned towards these types, we badly want to use florges cuz it won't get much opportunity otherwise
